# Analise de Sentimento com LLM + Engenharia de Prompt

Utilizar o GPT-5-Nano ou Ollama rodando na sua GPU (https://ollama.com/) para fazer análise de sentimentos do IMDB.
Faça uma amostragem balanceada do dataset (sugestão: 10%)

É um enunciado bem livre e vamos acompanhando durante a semana em função dos resultados parciais que vocês conseguem fazer.

Comparar zero-shot com alguma técnica de engenharia de prompts (CoT, self-consistency, etc.).

# Imports

In [ ]:
import os
import sys
import faiss
import time
import logging
import numpy as np
from openai import OpenAI
from typing import List, Dict

# Dados

In [ ]:
from datasets import load_dataset
from collections import Counter

# Carregamento do dataset IMDB
imdb_dic = load_dataset("stanfordnlp/imdb", token=False)

/Users/mabosco/Desktop/mestrado/IA368HH/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# exibindo algumas amostras do conjunto de treino
# exemplos label 0 são críticas negativas e exemplos label 1 são críticas positivas
# exibindo 5 amostras de cada
imdb_dic_0 = imdb_dic['train'].filter(lambda example: example['label'] == 0)
imdb_dic_1 = imdb_dic['train'].filter(lambda example: example['label'] == 1)

# amostras 0
print("Amostras com rótulo 0:")
for i in range(3):
    print(f"Amostra {i+1}:")
    print("Texto:", imdb_dic_0['text'][i])
    print("Rótulo:", imdb_dic_0['label'][i])
    print("-")

# amostras 1
print("Amostras com rótulo 1:")
for i in range(3):
    print(f"Amostra {i+1}:")
    print("Texto:", imdb_dic_1['text'][i])
    print("Rótulo:", imdb_dic_1['label'][i])
    print("-")

Amostras com rótulo 0:
Amostra 1:
Texto: I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and n

In [ ]:
# contar amostras positivas e negativas no treino
labels = imdb_dic['train']['label']
num_positivas = sum(1 for l in labels if l == 1)
num_negativas = sum(1 for l in labels if l == 0)
print(f"Amostras positivas: {num_positivas}")
print(f"Amostras negativas: {num_negativas}")

Amostras positivas: 12500
Amostras negativas: 12500


In [ ]:
from datasets import concatenate_datasets

# Calcular tamanhos das amostras
amostra_tamanho_treino = 25
amostra_tamanho_teste = 250

print(f"Tamanho da amostra de treino por classe: {amostra_tamanho_treino}")
print(f"Tamanho da amostra de teste por classe: {amostra_tamanho_teste}")

# ========== AMOSTRA DE TREINO ==========
# Amostras balanceadas do conjunto de treino
amostra_treino_0 = imdb_dic_0.shuffle(seed=42).select(range(amostra_tamanho_treino))
amostra_treino_1 = imdb_dic_1.shuffle(seed=42).select(range(amostra_tamanho_treino))

# Concatenar as amostras balanceadas de treino
imdb_amostra_treino = concatenate_datasets([amostra_treino_0, amostra_treino_1]).shuffle(seed=42)

print(f"\n=== CONJUNTO DE TREINO ===")
print(f"Tamanho da amostra de treino: {len(imdb_amostra_treino)}")
print(f"Distribuição do treino: {Counter(imdb_amostra_treino['label'])}")
print(f"Tamanho médio das amostras de treino: {np.mean([len(x.split()) for x in imdb_amostra_treino['text']]):.2f} palavras")
print(f"Quantidade de caracteres medio das amostras de treino: {np.mean([len(x) for x in imdb_amostra_treino['text']]):.2f} caracteres")


# ========== AMOSTRA DE TESTE ==========
# Filtrar exemplos do conjunto de teste por classe
imdb_test_0 = imdb_dic['test'].filter(lambda example: example['label'] == 0)
imdb_test_1 = imdb_dic['test'].filter(lambda example: example['label'] == 1)

# Amostras balanceadas do conjunto de teste
amostra_teste_0 = imdb_test_0.shuffle(seed=42).select(range(amostra_tamanho_teste))
amostra_teste_1 = imdb_test_1.shuffle(seed=42).select(range(amostra_tamanho_teste))

# Concatenar as amostras balanceadas de teste
imdb_amostra_teste = concatenate_datasets([amostra_teste_0, amostra_teste_1]).shuffle(seed=42)

print(f"\n=== CONJUNTO DE TESTE ===")
print(f"Tamanho da amostra de teste: {len(imdb_amostra_teste)}")
print(f"Distribuição do teste: {Counter(imdb_amostra_teste['label'])}")
print(f"Tamanho médio das amostras de teste: {np.mean([len(x.split()) for x in imdb_amostra_teste['text']]):.2f} palavras")
print(f"Quantidade de caracteres medio das amostras de teste: {np.mean([len(x) for x in imdb_amostra_teste['text']]):.2f} caracteres")

Tamanho da amostra de treino por classe: 25
Tamanho da amostra de teste por classe: 250

=== CONJUNTO DE TREINO ===
Tamanho da amostra de treino: 50
Distribuição do treino: Counter({0: 25, 1: 25})
Tamanho médio das amostras de treino: 230.20 palavras
Quantidade de caracteres medio das amostras de treino: 1301.32 caracteres

=== CONJUNTO DE TESTE ===
Tamanho da amostra de teste: 500
Distribuição do teste: Counter({1: 250, 0: 250})
Tamanho médio das amostras de teste: 226.93 palavras
Quantidade de caracteres medio das amostras de teste: 1284.22 caracteres


In [ ]:
imdb_amostra_teste[0]

{'text': 'Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unbeliaveble but you have to see it really!!!! P.s watch the carrot',
 'label': 1}

# Análise I: sem engenharia de prompt

In [ ]:
def generate_prompt(text: str, max_length: int = 5000) -> str:
    prompt = """Classify whether the following text from a movie review is positive or negative:""" + text + """
    """
    return prompt[:max_length]


In [ ]:
generate_prompt(imdb_amostra_balanceada[0]['text'])

'Classify whether the following text from a movie review is positive or negative:I\'m not sure under what circumstances director Visconti decided to film James Cain\'s novel "The Postman Always Rings Twice" (I\'m not even sure if Viscounti acquired the book\'s rights), but the resulting movie is definitely interesting. It is not the best version of Cain\'s story (I like the 1981 version best), but thanks to Visconti\'s excellent direction and the casting of Clara Calamai and Massimo Girotti (a very sensual couple), it is a must for noir fans. Visconti mixes neorealism with noir sensibilities to great effect. The film is not perfect, though. My main complaint is that the film is a little too long for its own good; the story moves at a very slow pace (I don\'t think Visconti was very good at editing his films). I think film noir works better with a short running time. Fortunately, Calamai and Girotti are magnetic actors that keep the viewer interested. Anyhow, as much as I like this film

In [ ]:
# Ler a chave da API do arquivo 'openai_key.txt'
with open('openai-key.txt', 'r') as f:
    key = f.read().strip()

# Configurar cliente OpenAI moderno
from openai import OpenAI
client = OpenAI(api_key=key)

In [ ]:
def get_answer(pergunta: dict, prompt_function: callable) -> Dict:
    # Gera o prompt
    prompt = prompt_function(pergunta['text'])

    t_inicial = time.time()  # Inicia o cronômetro

    # Gera a resposta
    response = client.responses.create(
        model="gpt-5-nano",
        input=[
          {
            "role": "developer",
            "content": [
              {
                "type": "input_text",
                "text": """
                You are a sentiment analysis model.
                The task is to classify whether a movie review is positive or negative.
                Return 1 if the review is positive and 0 if it is negative. Don't return any other value.
                """
              }
            ]
          },
          {
            "role": "user",
            "content": [
              {
                "type": "input_text",
                "text": prompt
              }
            ]
          }
        ],
        text={
          "format": {
            "type": "text"
          },
          "verbosity": "low"
        },
        reasoning={
          "effort": "minimal"
        },
        tools=[],
        store=True
    )

    t_final = time.time()  # Finaliza o cronômetro
    tempo_total = t_final - t_inicial  # Calcula o tempo total

    resposta = response.output[1].content[0].text.strip()  # Retorna a resposta

    resposta_dict = {
        'text': pergunta['text'],
        'label': pergunta['label'],
        'answer': resposta,
        'correct': int(resposta) == pergunta['label'],  # Verifica se a resposta está correta
        'time': tempo_total  # Adiciona o tempo total à resposta
    }

    return resposta_dict

In [ ]:
get_answer(imdb_amostra_balanceada[0], prompt_function=generate_prompt)

{'text': 'I\'m not sure under what circumstances director Visconti decided to film James Cain\'s novel "The Postman Always Rings Twice" (I\'m not even sure if Viscounti acquired the book\'s rights), but the resulting movie is definitely interesting. It is not the best version of Cain\'s story (I like the 1981 version best), but thanks to Visconti\'s excellent direction and the casting of Clara Calamai and Massimo Girotti (a very sensual couple), it is a must for noir fans. Visconti mixes neorealism with noir sensibilities to great effect. The film is not perfect, though. My main complaint is that the film is a little too long for its own good; the story moves at a very slow pace (I don\'t think Visconti was very good at editing his films). I think film noir works better with a short running time. Fortunately, Calamai and Girotti are magnetic actors that keep the viewer interested. Anyhow, as much as I like this film and the remakes, I think no one has made the definitive version of Cai

In [ ]:
from tqdm import tqdm
from datasets import Dataset

def aplica_get_answer(dataset: Dataset, prompt_function: callable) -> List[Dict]:
    # Aplicar get_answer em todas as amostras da amostra balanceada

    #respostas = [get_answer(exemplo, prompt_function=prompt_function) for exemplo in tqdm(dataset)]
    respostas = []
    for i,exemplo in enumerate(tqdm(dataset)):
        respostas.append(get_answer(exemplo, prompt_function=prompt_function))
        # imprimir a cada 50 exemplos
        if (i + 1) % 50 == 0:
            print(f"Processados {i + 1}/{len(dataset)} exemplos")

    # Salvar resultados em um novo Dataset HuggingFace
    respostas_dataset = Dataset.from_list(respostas)

    return respostas_dataset

In [ ]:
resultados = aplica_get_answer(imdb_amostra_teste, prompt_function=generate_prompt)

 10%|█         | 50/500 [01:40<15:31,  2.07s/it]

Processados 50/500 exemplos


 20%|██        | 100/500 [03:20<11:43,  1.76s/it]

Processados 100/500 exemplos


 30%|███       | 150/500 [05:04<12:25,  2.13s/it]

...

Processados 400/500 exemplos


 90%|█████████ | 450/500 [15:18<01:47,  2.16s/it]

Processados 450/500 exemplos


100%|██████████| 500/500 [17:00<00:00,  2.04s/it]

Processados 500/500 exemplos


In [ ]:
# Salvar o dataset de resultados em um arquivo JSON
resultados.to_json('09. Aula 8_9 - GPT 3/resultados_zero_shot.json')

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 118.50ba/s]


681748

# Análise II: com CoT

In [ ]:
def generate_prompt_cot(text: str, max_length: int = 10000) -> str:
    inicio = """Classify if the following text from a movie review is positive or negative.\n"""

    cot = """
    Exemples of reasoning formats:
    Review: I loved this movie! The acting was great and the story kept me engaged from start to finish. I highly recommend it.
    Reasoning: The words "loved", "great", and "highly recommend" indicate a positive sentiment towards the movie. The reviewer also mentions the acting and story as positive aspects.
    Final answer: 1
    \n
    Review: This movie was terrible. The plot made no sense and the acting was awful. I would not recommend it to anyone.
    Reasoning: The words "terrible", "made no sense", "awful", and "not recommend" indicate a negative sentiment towards the movie. The reviewer also mentions the plot and acting as negative aspects.
    Final answer: 0
    """

    final_instruction = """
    Review:
    """

    prompt = inicio + cot + final_instruction + text

    return prompt[:max_length]

In [ ]:
generate_prompt_cot(imdb_amostra_teste[0]['text'])

'Classify if the following text from a movie review is positive or negative.\n\n    Exemples of reasoning formats:\n    Review: I loved this movie! The acting was great and the story kept me engaged from start to finish. I highly recommend it.\n    Reasoning: The words "loved", "great", and "highly recommend" indicate a positive sentiment towards the movie. The reviewer also mentions the acting and story as positive aspects.\n    Final answer: 1\n    \n\n    Review: This movie was terrible. The plot made no sense and the acting was awful. I would not recommend it to anyone.\n    Reasoning: The words "terrible", "made no sense", "awful", and "not recommend" indicate a negative sentiment towards the movie. The reviewer also mentions the plot and acting as negative aspects.\n    Final answer: 0\n    \n    Review:\n    Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unbeliaveble but you have to see it really!!!! P.s watch the c

In [ ]:
resultados_cot = aplica_get_answer(imdb_amostra_teste, prompt_function=generate_prompt_cot)

 10%|█         | 50/500 [01:32<11:07,  1.48s/it]

Processados 50/500 exemplos


 20%|██        | 100/500 [03:07<10:45,  1.61s/it]

Processados 100/500 exemplos


 30%|███       | 150/500 [04:40<14:08,  2.43s/it]

...

Processados 400/500 exemplos


 90%|█████████ | 450/500 [14:23<01:46,  2.14s/it]

Processados 450/500 exemplos


100%|██████████| 500/500 [16:06<00:00,  1.93s/it]

Processados 500/500 exemplos


In [ ]:
resultados_cot.to_json('09. Aula 8_9 - GPT 3/resultados_cot.json')

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 66.34ba/s]


681740

# Analise III: Few-Shot

In [ ]:
# Selecionar um exemplo com label positiva (1) no dataset de treino
exemplo_positivo = next(ex for ex in imdb_amostra_treino if ex['label'] == 1)
print("Exemplo positivo do treino:")
print("Texto:", exemplo_positivo['text'])
print("Label:", exemplo_positivo['label'])

exemplo_negativo = next(ex for ex in imdb_amostra_treino if ex['label'] == 0)
print("\nExemplo negativo do treino:")
print("Texto:", exemplo_negativo['text'])
print("Label:", exemplo_negativo['label'])

Exemplo positivo do treino:
Texto: I saw this film at the Rhode Island International Film Festival and was completely blown away. The structure and execution of the film was fantastic...I know it won't, but it really deserves an Oscar nod. Cal and Andre were phenomenal as the two disturbed classmates. Yes, the film is very controversial and I can see a lot of people having a lot of problems with it, as it deals with school shooting and especially makes you identify with the killers. However, despite its harsh and blunt subject matter, Zero Day is SO worth watching. I'm looking forward to it coming out on video so I can buy it - it's very, very good. Very powerful and intense...the end shooting sequence leaves you speechless because it's almost too realistic. Their uncertainty, the "recordings", the footage and panic of the students, totally indescribable. I really hope it gets the attention it deserves. It's done in the same format as The Blair Witch Project, handheld camera, made to a

In [ ]:
exemples = imdb_amostra_treino.select(range(4))

def generate_prompt_few_shot(text, max_lenght=10000):
    inicio = """Classify if the following text from a movie review is positive or negative.\n"""

    exemplos = """Examples of positive reviews:\n"""

    for ex in exemples.filter(lambda x: x['label'] == 1):
        exemplos += f"- {ex['text']}\n"

    exemplos += """Examples of negative reviews:\n"""

    for ex in exemples.filter(lambda x: x['label'] == 0):
        exemplos += f"- {ex['text']}\n"

    exemplos += """Now analyze the sentiment of the following review:\n"""

    prompt = inicio + exemplos + text

    return prompt[:max_lenght]

In [ ]:
generate_prompt_few_shot(imdb_amostra_teste[0]['text'])

'Classify if the following text from a movie review is positive or negative.\nExamples of positivereviews:\n- I saw this film at the Rhode Island International Film Festival and was completely blown away. The structure and execution of the film was fantastic...I know it won\'t, but it really deserves an Oscar nod. Cal and Andre were phenomenal as the two disturbed classmates. Yes, the film is very controversial and I can see a lot of people having a lot of problems with it, as it deals with school shooting and especially makes you identify with the killers. However, despite its harsh and blunt subject matter, Zero Day is SO worth watching. I\'m looking forward to it coming out on video so I can buy it - it\'s very, very good. Very powerful and intense...the end shooting sequence leaves you speechless because it\'s almost too realistic. Their uncertainty, the "recordings", the footage and panic of the students, totally indescribable. I really hope it gets the attention it deserves. It\'

In [ ]:
get_answer(imdb_amostra_teste[0], prompt_function=generate_prompt_few_shot)

{'text': 'Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unbeliaveble but you have to see it really!!!! P.s watch the carrot',
 'label': 1,
 'answer': '0',
 'correct': False,
 'time': 1.7009499073028564}

In [ ]:
resultados_few = aplica_get_answer(imdb_amostra_teste, prompt_function=generate_prompt_few_shot)

 10%|█         | 50/500 [01:40<13:11,  1.76s/it]

Processados 50/500 exemplos


 20%|██        | 100/500 [03:23<14:54,  2.24s/it]

Processados 100/500 exemplos


 30%|███       | 150/500 [05:05<12:24,  2.13s/it]

...

Processados 400/500 exemplos


 90%|█████████ | 450/500 [14:58<01:49,  2.18s/it]

Processados 450/500 exemplos


100%|██████████| 500/500 [16:30<00:00,  1.98s/it]

Processados 500/500 exemplos


In [ ]:
resultados_few.to_json('09. Aula 8_9 - GPT 3/resultados_few.json')

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 121.96ba/s]


681740

# Avaliação

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def get_results(respostas):
    # Extrair rótulos verdadeiros e respostas do modelo
    y_true = respostas['label']
    y_pred = [int(ans) for ans in respostas['answer']]

    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, digits=4))

    # Matriz de confusão
    print("\nMatriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))

In [ ]:
print("Resultados do zero-shot:")
get_results(resultados)

Resultados do zero-shot:

Classification Report:
              precision    recall  f1-score   support

           0     0.9091    0.9600    0.9339       250
           1     0.9576    0.9040    0.9300       250

    accuracy                         0.9320       500
   macro avg     0.9334    0.9320    0.9319       500
weighted avg     0.9334    0.9320    0.9319       500


Matriz de Confusão:
[[240  10]
 [ 24 226]]


In [ ]:
print("Resultados do CoT:")
get_results(resultados_cot)

Resultados do CoT:

Classification Report:
              precision    recall  f1-score   support

           0     0.8836    0.9720    0.9257       250
           1     0.9689    0.8720    0.9179       250

    accuracy                         0.9220       500
   macro avg     0.9263    0.9220    0.9218       500
weighted avg     0.9263    0.9220    0.9218       500


Matriz de Confusão:
[[243   7]
 [ 32 218]]


In [ ]:
print("Resultados do Few-Shot:")
get_results(resultados_few)

Resultados do Few-Shot:

Classification Report:
              precision    recall  f1-score   support

           0     0.9104    0.9760    0.9421       250
           1     0.9741    0.9040    0.9378       250

    accuracy                         0.9400       500
   macro avg     0.9423    0.9400    0.9399       500
weighted avg     0.9423    0.9400    0.9399       500


Matriz de Confusão:
[[244   6]
 [ 24 226]]


# Exemplo de erros

In [ ]:
erros = resultados.filter(lambda x: not x['correct'])
print(f"Quantidade de erros: {len(erros)}\n")
print("Exemplos de erros:")
for i in range(3):
    print(f"Amostra {i+1}:")
    print("Texto:", erros['text'][i])
    print("Label:", erros['label'][i])
    print("Resposta do Modelo:", erros['answer'][i])
    print("-")
print("Tempo médio de resposta do modelo: {:.2f} segundos".format(np.mean(resultados['time'])))  # Calcula o tempo médio de resposta do modelo

Filter: 100%|██████████| 500/500 [00:00<00:00, 81209.42 examples/s]

Quantidade de erros: 34

Exemplos de erros:
Amostra 1:
Texto: I first saw this movie at a Saturday matinee when I was very young. I thought it was cool and often thought about it. Well I finally resaw it on DVD. It was still very entertaining but in a different way. It has to rank as one of the goofiest, campiest, 1950's sci-fi movies. It seemed filled with stock military footage. The dialogue is stilted and effects are crude. There is one line of dialogue that had me in stitches. The line Jeff Morrow says while on the beach with the babe. Rent it if you need a movie to watch with a bunch of drunken friends. It is a classic.
Label: 0
Resposta do Modelo: 1
-
Amostra 2:
Texto: My friends and I rented this movie mistaking it for another one about skateboarding. Watchin Steve Guttenberg as an action hero is hilarious. The movie is so incredibly predictable and over the top that it ended up being a laugh fest. Even though I gave it a 1/10 this movie should be seen especially if u manage to 

In [ ]:
erros_cot = resultados_cot.filter(lambda x: not x['correct'])
print(f"Quantidade de erros: {len(erros_cot)}\n")
print("Exemplos de erros:")
for i in range(3):
    print(f"Amostra {i+1}:")
    print("Texto:", erros_cot['text'][i])
    print("Label:", erros_cot['label'][i])
    print("Resposta do Modelo:", erros_cot['answer'][i])
    print("-")
print("Tempo médio de resposta do modelo: {:.2f} segundos".format(np.mean(resultados_cot['time'])))  # Calcula o tempo médio de resposta do modelo

Filter: 100%|██████████| 500/500 [00:00<00:00, 79188.61 examples/s]

Quantidade de erros: 39

Exemplos de erros:
Amostra 1:
Texto: I first saw this movie at a Saturday matinee when I was very young. I thought it was cool and often thought about it. Well I finally resaw it on DVD. It was still very entertaining but in a different way. It has to rank as one of the goofiest, campiest, 1950's sci-fi movies. It seemed filled with stock military footage. The dialogue is stilted and effects are crude. There is one line of dialogue that had me in stitches. The line Jeff Morrow says while on the beach with the babe. Rent it if you need a movie to watch with a bunch of drunken friends. It is a classic.
Label: 0
Resposta do Modelo: 1
-
Amostra 2:
Texto: The animal-eating (geek) scenes were not as bad as you would think. After having watched Mondo Cane and Mondo Magic, these scenes are average. The grossest one was when the guy ate the head off the mouse. But they were so fast and few that they didn't bother me.<br /><br />Otherwise, the film was just sort of inter

In [ ]:
erros_few = resultados_few.filter(lambda x: not x['correct'])
print(f"Quantidade de erros: {len(erros_few)}\n")
print("Exemplos de erros:")
for i in range(3):
    print(f"Amostra {i+1}:")
    print("Texto:", erros_few['text'][i])
    print("Label:", erros_few['label'][i])
    print("Resposta do Modelo:", erros_few['answer'][i])
    print("-")
print("Tempo médio de resposta do modelo: {:.2f} segundos".format(np.mean(resultados_few['time'])))  # Calcula o tempo médio de resposta do modelo

Filter: 100%|██████████| 500/500 [00:00<00:00, 88323.45 examples/s]

Quantidade de erros: 30

Exemplos de erros:
Amostra 1:
Texto: Worst horror film ever but funniest film ever rolled in one you have got to see this film it is so cheap it is unbeliaveble but you have to see it really!!!! P.s watch the carrot
Label: 1
Resposta do Modelo: 0
-
Amostra 2:
Texto: I first saw this movie at a Saturday matinee when I was very young. I thought it was cool and often thought about it. Well I finally resaw it on DVD. It was still very entertaining but in a different way. It has to rank as one of the goofiest, campiest, 1950's sci-fi movies. It seemed filled with stock military footage. The dialogue is stilted and effects are crude. There is one line of dialogue that had me in stitches. The line Jeff Morrow says while on the beach with the babe. Rent it if you need a movie to watch with a bunch of drunken friends. It is a classic.
Label: 0
Resposta do Modelo: 1
-
Amostra 3:
Texto: The 1978 adaptation had all the ingredients of a potentially wonderful film. It is bas